In [7]:
# Task Scheduler 

###### TODO
###### I could not reach the working solution with heap.
###### Probably I can use the similar solution to "1353. Maximum Number of Events That Can Be Attended"
###### https://leetcode.com/problems/maximum-number-of-events-that-can-be-attended/submissions/1221168417/?envType=list&envId=pfzmx095

# You are given an array of CPU tasks, each represented by letters A to Z, and a cooling time, n.
# Each cycle or interval allows the completion of one task.
# Tasks can be completed in any order, but there's a constraint:
# identical tasks must be separated by at least n intervals due to cooling time.

# Return the minimum number of intervals required to complete all tasks.


'''
# Interface
def min_num_intervals_for_completion(tasks, cooling_period) -> return num cycles (non-negative int)

tasks = [A, B, A], 2
tasks = [B, A, B], 2


# Examples

[A, B, A], 2
A B - A B    ----------> return 5

A - -
B - - 
A - -


AAABBCCCC
A - - A - - A
B - - B
C - - C - - C - - C
->
C A B C A B C A - C

AAABBCCCCDDD
C B A C B A C - A C

n = 2
x = 100
a = 100
b = 100
c = 100
d = 100

x a b c d x a b c d

n = 2
AAABBCCCC
->
A:3->2(next t=4~)
B:2->1(next t=5~)
C:4->3(next t=3~)->2(next t=6~)

0 1 2 3 
C A B C

time = len(tasks) + n * len(tasks) * len(tasks) = N * T**2
space = 
'''

from collections import defaultdict

class Solution:
    def min_num_intervals_for_completion(self, tasks, cooling):
        self.task_to_cnt = defaultdict(int)
        self.task_to_timestamp_after_cooling = {}
        for task in tasks:
            self.task_to_cnt[task] += 1
            self.task_to_timestamp_after_cooling[task] = 0
    
        t = 0
        while 0 < sum(self.task_to_cnt.values()):
            # print(t, self.task_to_cnt)
            # print(t, self.task_to_timestamp_after_cooling)
            task = self.find_task(t)
            # print("t=", t, ", do task=",task)
            
            if task:
                self.task_to_cnt[task] -= 1
                self.task_to_timestamp_after_cooling[task] = t + cooling + 1   
            t += 1

        return t
    
    def find_task(self, t):

        possible_tasks = [] # (task, rem_cmt)
        for task, rem_cnt in self.task_to_cnt.items():
            if rem_cnt == 0:
                continue
            if t < self.task_to_timestamp_after_cooling[task]:
                continue
            possible_tasks.append((task, rem_cnt))

        return max(possible_tasks, key=lambda x: x[1])[0] if 0 < len(possible_tasks) else None

assert Solution().min_num_intervals_for_completion("ABA", 2) == 4 # ab-a
assert Solution().min_num_intervals_for_completion("BBB", 2) == 7
assert Solution().min_num_intervals_for_completion("ABCDABCD", 2) == 8


# ========================================================================================================
# Re-implement with heap
# Heap solution (1)

import heapq
from collections import Counter

class Solution:
    def min_num_intervals_for_completion(self, tasks, cooling):
        self.cooling = cooling
        # Use min heap of (timestamp_after_cooling, - task cnt, task_id)
        self.tasks = [] 
        for task_id, cnt in Counter(tasks).items():
            self.tasks.append((0, -cnt, task_id))
        heapq.heapify(self.tasks)
        
        t = 0
        while 0 < len(self.tasks):
            self.do_task(t)
            t += 1

        return t
    
    def do_task(self, t):
        ealiest_t_for_next_task = self.tasks[0][0]
        if t < ealiest_t_for_next_task:
            return None
        out = heapq.heappop(self.tasks)
        task_cnt, task_id = - out[1], out[2]

        if 2 <= task_cnt:
            t_after_cooling = t + self.cooling + 1
            rem_task_cnt = task_cnt - 1
            heapq.heappush(self.tasks, (t_after_cooling, -rem_task_cnt, task_id))




# #=====================================================================================================
# # After I read the answer again, I noticed that I don't need iterate with t.
# # (This part can be the bottleneck, if the cooling period is really long, e.g. ("AA", 100000000)
# # Heap solution (2)

# class Solution:
#     def min_num_intervals_for_completion(self, tasks, cooling):
#         self.cooling = cooling
#         # Use min heap of (- task cnt, task_id)
#         self.tasks = [] 
#         for task_id, cnt in Counter(tasks).items():
#             self.tasks.append((1, -cnt, task_id))
#         heapq.heapify(self.tasks)
        
#         current_time = 0
#         while 0 < len(self.tasks):
#             current_time += 1
#             print(current_time, self.tasks)
#             current_time = self.do_next_task(current_time)
#         return current_time

#     # Returns the time when next task is finished.
#     def do_next_task(self, current_time):
#         out = heapq.heappop(self.tasks)
#         task_workable_from, task_cnt, task_id = out[0], - out[1], out[2]
#         task_finished_time = max(current_time, task_workable_from)

#         if 2 <= task_cnt:
#             t_after_cooling = task_finished_time + self.cooling + 1
#             rem_task_cnt = task_cnt - 1
#             heapq.heappush(self.tasks, (t_after_cooling, -rem_task_cnt, task_id))

#         return task_finished_time

'''
Assuming that there are K task_ids, N unique tasks,
time  = N + N*logK
space = K

'''

assert Solution().min_num_intervals_for_completion("ABA", 2) == 4 # ab-a
assert Solution().min_num_intervals_for_completion("BBB", 2) == 7
assert Solution().min_num_intervals_for_completion("ABCDABCD", 2) == 8

# In this case, below does not work.
# According to the current time, the next task to do differs.
assert Solution().min_num_intervals_for_completion(["A","A","A","A","A","A","B","C","D","E","F","G"], 1) == 8

# t = 1: [(1, -6, 'A'), (1, -1, 'B'), (1, -1, 'C'), (1, -1, 'D'), (1, -1, 'E'), (1, -1, 'F'), (1, -1, 'G')]
# t = 2: [(1, -1, 'B'), (1, -1, 'D'), (1, -1, 'C'), (1, -1, 'G'), (1, -1, 'E'), (1, -1, 'F'), (3, -5, 'A')]
# t = 3: [(1, -1, 'C'), (1, -1, 'D'), (1, -1, 'F'), (1, -1, 'G'), (1, -1, 'E'), (3, -5, 'A')]
# t = 4: [(1, -1, 'D'), (1, -1, 'E'), (1, -1, 'F'), (1, -1, 'G'), (3, -5, 'A')]
# t = 5: [(1, -1, 'E'), (1, -1, 'G'), (1, -1, 'F'), (3, -5, 'A')]
# t = 6: [(1, -1, 'F'), (1, -1, 'G'), (3, -5, 'A')]
# t = 7: [(1, -1, 'G'), (3, -5, 'A')]
# t = 8: [(3, -5, 'A')]
# t = 9: [(10, -4, 'A')]
# t = 11: [(12, -3, 'A')]
# t = 13: [(14, -2, 'A')]
# t = 15: [(16, -1, 'A')]

# If t = 4, I would do (3, -5, 'A'), which has more tasks rather than (1, -1, 'D').



AssertionError: 

In [8]:
# Maximum Number of Events That Can Be Attended

# You are given an array of events where events[i] = [startDayi, endDayi]. Every event i starts at startDayi and ends at endDayi.
# You can attend an event i at any day d where startTimei <= d <= endTimei. You can only attend one event at any time d.
# Return the maximum number of events you can attend.

'''

# Example

[1,5],[1,5],[1,5],[2,3],[2,3]

1------5
1------5
1------5
  2-3
  2-3


curr_day = 1



sort events

heap = [] # (end_date, start_date)

for each day,
   for each event which starts on that day, push it into heap
   pop out event
       if it is invalid (out.end_date < curr.start_date), ignore it.
       otherwise, cnt += 1
'''

from heapq import heappush, heappop, heapify

# Impl
def attendable_cnt(events):
    events.sort()
    cnt = 0
    
    started_events = [] # (end, start)

    i = 0
    curr_day = 1
    while i < len(events) or 0 < len(started_events):
        # Push all the events started today into the heap `started_events`.
        while i < len(events) and events[i][0] <= curr_day:
            event = events[i]
            heappush(started_events, (event[1], event[0]))
            i += 1

        # Pop out all the outdated events from heap.
        while 0 < len(started_events) and started_events[0][0] < curr_day:
            heappop(started_events)

        # Pop out an event attended today.
        if 0 < len(started_events):
            heappop(started_events)
            cnt += 1

        curr_day += 1
    return cnt


'''
time  = NlogN + max(endDate)
space = N

'''

# Test
print(attendable_cnt([[2,2],[1,2], [1,2]]))
assert attendable_cnt([[2,2],[1,2], [1,2]]) == 2
assert attendable_cnt([[2,2],[1,2]]) == 2
assert attendable_cnt([]) == 0
assert attendable_cnt([[1,5],[1,5],[1,5],[2,3],[2,3]]) == 5

2


In [ ]:
# Reorganize string

'''
aab -> aba
{a: 2, b: 1}
-> use "a" (the most cnt)
{a: 1, b: 1}
-> use "b" (not a)
{a: 1, b: 0}
-> use "a" (the most cnt)

Use min heap.
h = [] # (- cnt, char)

count each char.
build heap (-cnt, char)

keep track of chars = []
while 0 < len(heap)
    pop out from heap(1)
    if out_char != prev_char,
        decrement cnt and push it back.
        append to chars
    otherwise,
        pop one more(2). if there is not any, return ""
        append (2) to chars
        push back (1) and cnt-decrmented (2)

joined chars

Assuming N=len(s) and K=nunique(s),
time  = N log K
space = K

'''

class Solution:
    def reorganizeString(self, s: str) -> str:
        # (-rem_cnt, char)
        rem_chars = [(-cnt, char) for char, cnt in Counter(s).items()]
        heapq.heapify(rem_chars)

        selected_chars = []

        while 0 < len(rem_chars):
            out = heapq.heappop(rem_chars)
            rem_cnt, char = - out[0], out[1]

            if len(selected_chars) == 0 or selected_chars[-1] != char:
                selected_chars.append(char)
                rem_cnt -= 1
                if 0 < rem_cnt:
                    heapq.heappush(rem_chars, (-rem_cnt, char))
            else:
                if len(rem_chars) == 0:
                    return ""
                out2 = heapq.heappop(rem_chars)
                rem_cnt2, char2 = - out2[0], out2[1]
                assert selected_chars[-1] != char2

                selected_chars.append(char2)

                heapq.heappush(rem_chars, (-rem_cnt, char))

                rem_cnt2 -= 1
                if 0 < rem_cnt2:
                    heapq.heappush(rem_chars, (-rem_cnt2, char2))
        
        return "".join(selected_chars)



In [20]:


# A string s is called happy if it satisfies the following conditions:

# s only contains the letters 'a', 'b', and 'c'.
# s does not contain any of "aaa", "bbb", or "ccc" as a substring.
# s contains at most a occurrences of the letter 'a'.
# s contains at most b occurrences of the letter 'b'.
# s contains at most c occurrences of the letter 'c'.
# Given three integers a, b, and c, return the longest possible happy string. If there are multiple longest happy strings, return any of them.
# If there is no such string, return the empty string "".

# A substring is a contiguous sequence of characters within a string.

'''
# interface
def longeset_happy_string(a, b, c) -> return string

- a, b, c is integer. 0 <= a,b,c.

# example
a,b,c = 2,3,4

aabbbcccc   x
aabccbbcc   o

a,b,c = 2,2,100
cc a cc b cc a cc b cc
{a2b2c99}-> {a2b2c98} -> {a1b2c98}
cc a cc

# algorithm

Track
- second_last_char, last_chars
- rem_char_to_cnt (list of (- frequency, char))
- chars

while len(rem_char_to_cnt):
    pop element. 
    if out = second_last_char = last_chars, pop again.
    append used one to char.
    decremnt frequency and push it back.
    for the other, push it back
return contanated chars.

time  = A + B + C
space = A + B + C
'''
# Impl
def longeset_happy_string(a, b, c): # a8, b1, c0
    # x(-8, a), (-1, b), (-7, a)
    rem_chars = [(-cnt, char) for char, cnt in [("a", a), ("b", b), ("c", c)] if 0 < cnt] # x(-8, a), (-1, b), x(-7, a),  (-6, a), ()
    heapq.heapify(rem_chars)
    chars = [] # a, a, b
    second_last_char, last_char = None, None # a, a

    while 0 < len(rem_chars):
        out1 = heapq.heappop(rem_chars)
        cnt1, char1 = - out1[0], out1[1] # 6, a

        #  
        if second_last_char == last_char == char1:
            if len(rem_chars) == 0:
                break
            out2 = heapq.heappop(rem_chars)
            cnt2, char2 = - out2[0], out2[1] # 1, b

            chars.append(char2)
            heapq.heappush(rem_chars, (-cnt1, char1))
            second_last_char, last_char = last_char, char2
            if 2 <= cnt2:
                heapq.heappush(rem_chars, (-cnt2+1, char2))
        else:
            chars.append(char1)
            if 2 <= cnt1:
                heapq.heappush(rem_chars, (-cnt1+1, char1))
            second_last_char, last_char = last_char, char1

    return "".join(chars)
    
# Test
print(longeset_happy_string(8,1,1))
assert longeset_happy_string(8,1,1) in ["aabaacaa", "aacaabaa"]
assert longeset_happy_string(8,1,0) == "aabaa"
assert longeset_happy_string(8,0,0) == "aa"
assert longeset_happy_string(0,0,0) == ""

aabaacaa
